In [26]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [27]:
# Define the KNN class
class KNN:
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        # TODO: Implement the fit method
        self.X = X
        self.y = y
        pass

    def predict(self, X):
        # TODO: Implement the predict method

        predictions = []

        for row in X:
            #if row in self.X:
            #    predictions.append([self.y[i] for i in range(len(self.y)) if np.allclose(y[i], row)][0])
            #else:
                #distances = [[self.compute_distance(row, self.X[i]), self.Y[i]] for i in range(len(self.X))]
                #orted_dis = sorted(distances)
                #topk = sorted_dis[:self.k]

                #classification = self.neighbor_majority(topk)

                #predictions.append(classification)
            distances = np.apply_along_axis(self.compute_distance, 1, self.X, row)
            dis_classifications = np.array([[distances[i], self.y[i]] for i in range(len(self.y))]) #np.concatenate((distances, self.y), axis=1)
            #sort_dis_class = np.sort(dis_classifications)
            #sort_dis_class = sorted(dis_classifications)
            sort_dis_class = dis_classifications[dis_classifications[:, 0].argsort()]

            topk = sort_dis_class[:self.k]
            classification = self.neighbor_majority(topk)

            predictions.append(classification)

        return np.array(predictions)

    def neighbor_majority(self, topk):
        #class_counts = Counter([classification for _, classification in topk])

        #most_frequent_classes = class_counts.most_common()

        #if len(most_frequent_classes) == 1:
        #    return most_frequent_classes[0][0]
        #else:
        #    if most_frequent_classes[0][1] > most_frequent_classes[1][1]:
        #        return most_frequent_classes[0][0]
        #    else:
        #        inv_sq_sums = {0:0, 1:0}
        #        for dis, classification in topk.items():
        #            inv_sq_sums[classification] += 1 / (dis ** 2)
        #            return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        class_counts = {1:0, 0:0}
        #inv_sq_sums = {1:0, 0:0}
        #print(topk)
        for dis, classification in topk:
            class_counts[classification] += 1
        #    inv_sq_sums[classification] += 1 / (dis ** 2)

        #if class_counts[1] != class_counts[0]:
            #print(f"Majority: {1 if class_counts[1] > class_counts[0] else 0}")
        #    return 1 if class_counts[1] > class_counts[0] else 0
        #else:
        #    return 1 if inv_sq_sums[1] > inv_sq_sums[0] else 0
        return class_counts[1] / self.k


        

    def compute_distance(self, X1, X2):
        # TODO: Implement distance computation based on self.distance_metric
        # Hint: Use numpy operations for efficient computation

        return np.linalg.norm(X1 - X2)

In [28]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    labels = train_data["Exited"].to_numpy(dtype=int)
    
    train_dropped = train_data.drop(columns=["id", "CustomerId", "Surname", "Geography", "Exited"])
    test_dropped = test_data.drop(columns=["id", "CustomerId", "Surname", "Geography"])

    train_encoded = pd.get_dummies(train_dropped, columns=["Gender"], dtype=int)
    test_encoded = pd.get_dummies(test_dropped, columns=["Gender"], dtype=int)

    for col_name in train_encoded.columns:
        max_val = max(train_encoded[col_name].max(), test_encoded[col_name].max())
        if max_val != 0:
            train_encoded[col_name] = train_encoded[col_name] / max_val
            test_encoded[col_name] = test_encoded[col_name] / max_val

    train_np = train_encoded.to_numpy(dtype=float)
    test_np = test_encoded.to_numpy(dtype=float)

    train_rank = np.linalg.matrix_rank(train_np)
    reduced_rank = train_rank - 2
    u, s, vt = np.linalg.svd(train_np, full_matrices=False)
    s = np.real(s[:train_rank])
    sig = np.diag(s)
    u = u[:, :reduced_rank]
    sig = sig[:reduced_rank,:reduced_rank]
    vt = vt[:reduced_rank]
    train_np = u @ sig @ vt

    test_rank = np.linalg.matrix_rank(test_np)
    reduced_rank = test_rank - 2
    u, s, vt = np.linalg.svd(test_np, full_matrices=False)
    s = np.real(s[:test_rank])
    sig = np.diag(s)
    u = u[:, :reduced_rank]
    sig = sig[:reduced_rank,:reduced_rank]
    vt = vt[:reduced_rank]
    test_np = u @ sig @ vt


    # TODO: Implement data preprocessing
    # Handle categorical variables, scale features, etc.
    return train_np, labels, test_np

In [29]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    # TODO: Implement cross-validation
    # Compute ROC AUC scores

    data_splits = np.array_split(X, n_splits)
    class_splits = np.array_split(y, n_splits)

    for i in range(n_splits):
        curr_test_data = data_splits[i]
        curr_test_classes = class_splits[i]

        
        curr_train_data = [data_splits[j] for j in range(n_splits) if j != i]
        curr_train_classes = [class_splits[i] for j in range(n_splits) if j != i]

        train = np.concatenate(curr_train_data)
        train_classes = np.concatenate(curr_train_classes)


        knn.fit(train, train_classes)

        test_prediction = knn.predict(curr_test_data)
        results = test_prediction == curr_test_classes
        num_right = sum([1 for t in results if t])
        num_wrong = sum([1 for t in results if not t])
        print(f"Num right: {num_right}. Num wrong: {num_wrong}")
        count = 0
        #for pred, actual in zip(test_prediction, curr_test_classes):
        #    print(f"Prediction: {pred}. Actual: {actual}.")
        #    if count == 100:
        #        break
        #    count += 1
    return "hi"



In [30]:
# Load and preprocess data
X, y, X_test = preprocess_data('data/train.csv', 'data/test.csv')

# Create and evaluate model
#knn = KNN(k=5, distance_metric='euclidean')

# Perform cross-validation
#cv_scores = cross_validate(X, y, knn)

#print("Cross-validation scores:", cv_scores)

# TODO: hyperparamters tuning
#for k_value in range(5, 21):
#    print(f"Value for k {k_value}")
#    knn = KNN(k=k_value, distance_metric="euclidean")
#    cross_validate(X, y, knn)


# TODO: Train on full dataset with optimal hyperparameters and make predictions on test set
knn = KNN(k=17, distance_metric="euclidean")
knn.fit(X, y)
test_predictions = knn.predict(X_test)

# Save test predictions
pd.DataFrame({'id': pd.read_csv('data/test.csv')['id'], 'Exited': test_predictions}).to_csv('submissions.csv', index=False)

Value for k 5
Num right: 2295. Num wrong: 705
Num right: 2326. Num wrong: 674
Num right: 2306. Num wrong: 694
Num right: 2286. Num wrong: 714
Num right: 2263. Num wrong: 737
Value for k 6
Num right: 2308. Num wrong: 692
Num right: 2318. Num wrong: 682
Num right: 2285. Num wrong: 715
Num right: 2284. Num wrong: 716
Num right: 2264. Num wrong: 736
Value for k 7
Num right: 2331. Num wrong: 669
Num right: 2347. Num wrong: 653
Num right: 2342. Num wrong: 658
Num right: 2334. Num wrong: 666
Num right: 2301. Num wrong: 699
Value for k 8
Num right: 2335. Num wrong: 665
Num right: 2361. Num wrong: 639
Num right: 2342. Num wrong: 658
Num right: 2328. Num wrong: 672
Num right: 2301. Num wrong: 699
Value for k 9
Num right: 2345. Num wrong: 655
Num right: 2376. Num wrong: 624
Num right: 2362. Num wrong: 638
Num right: 2373. Num wrong: 627
Num right: 2325. Num wrong: 675
Value for k 10
Num right: 2351. Num wrong: 649
Num right: 2383. Num wrong: 617
Num right: 2365. Num wrong: 635
Num right: 2361. Nu